## I) Instalando paquetes necesarios

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time

import pandas as pd

## II) Para obtener el número total de elecciones que se necesitan

In [2]:
 # Set antes de entrar a la página web #
service = Service()
options = webdriver.ChromeOptions()
options.add_argument( '--ignore-certificate-errors' )
options.add_argument( '--incognito' )  # Para que entre en modo incógnito

driver = webdriver.Chrome(service = service, options = options)
driver.maximize_window()

In [3]:
# Ingresando a la página web
enlace_base = "https://infogob.jne.gob.pe/Eleccion"
driver.set_page_load_timeout( 20 )
    
web = driver.get(enlace_base)
time.sleep(15)

In [4]:
# Para hacer selección en filtro "Tipo de proceso" = Elecciones presidenciales
filtro_proceso = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div')
filtro_proceso.click()
time.sleep(2)
    
# Mover hacia abajo en la lista desplegable tres veces
for _ in range(1):
    ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform()
    time.sleep(2)
    
# Seleccionar la opción actualmente resaltada
ActionChains(driver).send_keys(Keys.ENTER).perform()
time.sleep(2)

In [5]:
# Para obtener total de opciones en filtro "Elección"
filtro_eleccion = driver.find_element(By.XPATH,'//*[@id="IdEleccion"]')
select = Select(filtro_eleccion)
all_eleccion = select.options
total_eleccion = len(all_eleccion)

print(f"Total de elementos en la lista desplegable: {total_eleccion}")

Total de elementos en la lista desplegable: 26


In [10]:
total_eleccion

26

## III) Iniciando searching en la web

In [23]:
# Configuración del bucle
service = Service()
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
driver = webdriver.Chrome(service=service, options=options)
driver.maximize_window()

enlace_base = "https://infogob.jne.gob.pe/Eleccion"
driver.set_page_load_timeout(10)

In [ ]:
# Lugar para las tablas obtenidas
tables_list = []


i = total_eleccion + 1

for k in range(1,i):
    time.sleep(10)
    # Set antes de entrar a la página web
    driver.get(enlace_base)
    time.sleep(10)

    # Para hacer selección en filtro "Tipo de proceso"
    filtro_proceso = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div')
    filtro_proceso.click()
    time.sleep(2)

    # Mover hacia abajo en la lista desplegable una vez
    ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform()
    time.sleep(2)

    # Seleccionar la opción actualmente resaltada
    ActionChains(driver).send_keys(Keys.ENTER).perform()
    time.sleep(2)

    # Para hacer selección en filtro "Elección"
    filtro_eleccion = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    filtro_eleccion.click()
    time.sleep(2)

    # Mover hacia abajo en la lista desplegable k veces
    for _ in range(k):
        ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform()
        time.sleep(2) # lo cambié de 0.5 a 2 s.

    # Seleccionar la opción actualmente resaltada
    ActionChains(driver).send_keys(Keys.ENTER).perform()
    time.sleep(2) # lo cambié de 0.5 a 2 s.
    
    # Para ingresar a ver los datos de la elección seleccionada
    boton_datos = driver.find_element(By.XPATH, '//*[@id="btnVerDatos"]')
    boton_datos.click()
    time.sleep(10)

    ########### A partir de acá se va a la página para la obtención de datos ###########
    ########### Ingresar códigos dentro del bucle ###########
    
    
    # Para ingresar a ver la tabla de resultados de la elección seleccionada
    resultado_datos = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a')
    resultado_datos.click()
    time.sleep(10)

    
    # Leer el texto html de la tabla
    table_path = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]')
    table_html = table_path.get_attribute('innerHTML')
    table_data = pd.read_html(table_html)[0]
    
    # Agregar una columna 'Elecciones' con el valor de filtro_eleccion
    table_data['Elecciones'] = filtro_eleccion.text

    # Unir la tabla obtenida en la lista de tablas.
    tables_list.append(table_data)
    time.sleep(2)
    
    # Imprimir la tabla de la actual iteración
    print(f"Table Data for Iteration {k}:\n")
    print(table_data)
    print("\n" + "=" * 50 + "\n")
    
    time.sleep(2)

# Cerrar el navegador al final del bucle
driver.quit()

# Concatenar todas las tablas en un único DataFrame final
final_dataframe = pd.concat(tables_list, ignore_index=True)[['Elecciones', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]
print("\nFinal DataFrame:\n")

# Guardar el dataframe como un archivo Excel
final_dataframe.to_excel("resultados_elecciones.xlsx", index=False)

final_dataframe.head(10)